In [1]:
import pandas as pd
import numpy as np
import re

from pymorphy2 import MorphAnalyzer
from nltk import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
train_df = pd.read_csv('../dataset/RoboMed/train_data.csv')
test_df = pd.read_csv('../dataset/RoboMed/test_data.csv')

In [76]:
train_df[train_df.Диагноз == 'Преждевременная эякуляция']['Анамнез заболевания'].apply(print)

на фоне лечение видимого улучшения не отмечает, высыпания и зуд сохраняются. всвязи с отсутствием положительной динамики рекомендована выдача л/н.
Неделю назад в г. Нижний Новгород перенес оперативное удаление атером на мошонке (со слов).
Лидокаиновый тест положительный.
Проводит обследование по жалобам. Явилась по рез-там обследования
на ЛВН№ 218 205 370 403 с 28.07.16г по 01.08.16г по поводу ОРВИ (выдан на дому ООО медилюкс)- ДН-5. на фоне проводимой терапии (полоскание горла мирамистином, промывание носа аква-марис, Ксимелин спрей в нос) - отмечает  уменьшение с-ов ринофарингита, нормализацию Т тела. Рекомендованной Полидексой не пользовалась.
Направлен терапевтом. Готовится к оперативному лечению по исправлению искривленной носовой перегородки.  Эпизодически частый пульс (при нагрузках), снижается в покое. Это не беспокоит. Активно занимается спортом (играет в баскетбол). Физические нагрузки переносит хорошо. Боли в грудной клетке, нарушения ритма не беспокоят.
пациент настороже по

6789      None
59136     None
63475     None
92115     None
115600    None
146284    None
184550    None
195066    None
196366    None
199530    None
210898    None
222465    None
Name: Анамнез заболевания, dtype: object

In [60]:
test_df[(test_df['Диагноз'] == 'Острый вагинит') & (test_df['Пол'] == 1)]

,ID,Код_диагноза,Диагноз,Возраст,Пол,Общее состояние,аллергия,Анамнез заболевания,Внешний осмотр
3181,3181,группа_N номер_76 подномер_0,Острый вагинит,34,1,"Удовлетворительное. Температура тела - 36,6. П...",аллергии_неизвестны,нет_анамнеза,нет_осмотра
4844,4844,группа_N номер_76 подномер_0,Острый вагинит,36,1,общее_состояние_неизвестно,аллергии_неизвестны,"отмечает около 5-6 лет, с начала переходного в...",нет_осмотра
7603,7603,группа_N номер_76 подномер_0,Острый вагинит,58,1,общее_состояние_неизвестно,аллергии_неизвестны,около 2х недели,нет_осмотра
9487,9487,группа_N номер_76 подномер_0,Острый вагинит,22,1,Удовлетворительное,Без дополнений,нет_анамнеза,нет_осмотра
13293,13293,группа_N номер_76 подномер_0,Острый вагинит,25,1,общее_состояние_неизвестно,аллергии_неизвестны,Жалобы около недели,нет_осмотра
15308,15308,группа_N номер_76 подномер_0,Острый вагинит,33,1,общее_состояние_неизвестно,аллергии_неизвестны,"около 10 дней, была у ЛОРа 9.02.18 - был курс ...",нет_осмотра
15507,15507,группа_N номер_76 подномер_0,Острый вагинит,32,1,общее_состояние_неизвестно,аллергии_неизвестны,"МКБ в анамнезе около 45 лет, неоднократно отхо...",нет_осмотра
20843,20843,группа_N номер_76 подномер_0,Острый вагинит,27,1,общее_состояние_неизвестно,аллергии_неизвестны,Со слов боли беспокоят в течении 2 недель. Тра...,нет_осмотра
21606,21606,группа_N номер_76 подномер_0,Острый вагинит,32,1,общее_состояние_неизвестно,аллергии_неизвестны,"Вышеперечисленные жалобы в течение неск. лет, ...",нет_осмотра
31191,31191,группа_N номер_76 подномер_0,Острый вагинит,39,1,общее_состояние_неизвестно,аллергии_неизвестны,явилась по рез-там обследования,нет_осмотра


In [53]:
train_df[train_df['Анамнез заболевания'].str.len() < 100]

,ID,Код_диагноза,Диагноз,Возраст,Пол,Общее состояние,аллергия,Анамнез заболевания,Внешний осмотр,Revisit
0,0,группа_J номер_06 подномер_0,Острый ларингофарингит,29,2,общее_состояние_неизвестно,аллергии_неизвестны,нет_анамнеза,На жевательной поверхности 2.6- кариозная поло...,1
2,2,группа_L номер_23 подномер_9,"Аллергический контактный дерматит, причина не ...",63,2,общее_состояние_неизвестно,аллергии_неизвестны,"Болен около 2-х дней, когда появились описанны...",нет_осмотра,1
3,3,группа_N номер_48 подномер_1,Баланопостит,43,2,общее_состояние_неизвестно,не отягощен,нет_анамнеза,нет_осмотра,1
4,4,группа_Z номер_00 подномер_0,Общий медицинский осмотр,29,1,общее_состояние_неизвестно,аллергии_неизвестны,провела полное обследование для оперативного л...,нет_осмотра,1
5,5,группа_Z номер_01 подномер_,"Другие специальные осмотры и обследования лиц,...",50,2,удовлетворительное.,не отягощен,нет_анамнеза,нет_осмотра,1
6,6,группа_M номер_23 подномер_2,Поражение мениска в результате старого разрыва...,42,2,общее_состояние_неизвестно,аллергии_неизвестны,"проводит лечение, прибыла для коррекции назнач...",нет_осмотра,1
8,8,группа_N номер_77 подномер_1*,"Вагинит, вульвит и вульвовагинит при инфекцион...",28,2,Общее самочувствие удовлетворительное. Кожные ...,аллергии_неизвестны,нет_анамнеза,нет_осмотра,1
9,9,группа_H номер_60 подномер_,Наружный отит,38,1,общее_состояние_неизвестно,аллергии_неизвестны,Ранее к эндокринологу не обращалась и не обсле...,нет_осмотра,1
10,10,группа_H номер_52 подномер_2,Астигматизм,25,2,общее_состояние_неизвестно,аллергии_неизвестны,Вышеописанные жалобы беспокоят около 5 лет. Ле...,нет_осмотра,1
11,11,группа_K номер_30 подномер_,Диспепсия,51,1,общее_состояние_неизвестно,аллергии_неизвестны,Пришла по результатам обследования,нет_осмотра,0


In [4]:
def explain_diagnosis_code(series):
    regex = '([A-Z])(\d\d)(?:.(\d\*?))?'
    prefixes = ['группа_', 'номер_', 'подномер_']
    def get_explanation(string):
        matches = re.findall(regex, string)
        if len(matches) == 0:
            return 'неизвестный_диагноз'
        return [prefix + token for prefix, token in zip(prefixes, matches[0])]
    return series.apply(lambda string: ' '.join(get_explanation(string)))

In [5]:
train_df['Общее состояние'] = train_df['Общее состояние'].fillna('общее_состояние_неизвестно')
train_df['аллергия'] = train_df['аллергия'].fillna('аллергии_неизвестны')
train_df['Анамнез заболевания'] = train_df['Анамнез заболевания'].fillna('нет_анамнеза')
train_df['Внешний осмотр'] = train_df['Внешний осмотр'].fillna('нет_осмотра')
train_df['Код_диагноза'] = explain_diagnosis_code(train_df['Код_диагноза'])

test_df['Общее состояние'] = test_df['Общее состояние'].fillna('общее_состояние_неизвестно')
test_df['аллергия'] = test_df['аллергия'].fillna('аллергии_неизвестны')
test_df['Анамнез заболевания'] = test_df['Анамнез заболевания'].fillna('нет_анамнеза')
test_df['Внешний осмотр'] = test_df['Внешний осмотр'].fillna('нет_осмотра')
test_df['Код_диагноза'] = explain_diagnosis_code(test_df['Код_диагноза'])

In [6]:
train_size = train_df.shape[0]
print(train_size)

283086


In [7]:
# train_df = train_df.fillna('')
# test_df = test_df.fillna('')

In [8]:
re.sub(r"([.\!?,'/()])", ' \\1 ', 'Это самое обычное предложение, которое в том числе заканчивается!')

'Это самое обычное предложение ,  которое в том числе заканчивается ! '

In [9]:
train_dataframe, valid_dataframe = train_test_split(train_df, test_size=0.1,
                                                    stratify=train_df.Revisit, random_state=42)

In [24]:
def concat_features_ft(df, train=True):
    features = (
        df['Код_диагноза'].str.replace('.', '#') + ' | ' +
        df['Диагноз'] + ' | ' +
        'Возраст_' + df['Возраст'].astype(str) + ' | ' +
        'Пол_' + df['Пол'].astype(str) + ' | ' +
        df['Общее состояние'] + ' | ' +
        df['аллергия'] + ' | ' +
        df['Анамнез заболевания'] + ' | ' +
        df['Внешний осмотр']
    )
    
    text = features
    if train:
        text = '__label__' + df.Revisit.astype(str) + ' ' + text

    return text.str.lower().str.replace('\n', '').apply(lambda string: re.sub(r"([.\!?,'/()\-№:;])", ' \\1 ', string))

In [25]:
fasttext_train = concat_features_ft(train_dataframe)
fasttext_valid = concat_features_ft(valid_dataframe)
fasttext_test = concat_features_ft(test_df, train=False)

In [26]:
with open('fasttext_train.txt', 'w') as file:
    for line in fasttext_train:
        file.write(line + '\n')

In [27]:
with open('fasttext_valid.txt', 'w') as file:
    for line in fasttext_valid:
        file.write(line + '\n')

In [28]:
with open('fasttext_test.txt', 'w') as file:
    for line in fasttext_test:
        file.write(line + '\n')

In [29]:
! tail -n 10 fasttext_test.txt

группа_n номер_97 подномер_ | женское бесплодие | возраст_32 | пол_2 | общее самочувствие удовлетворительное .  кожные покровы и видимые слизистые обычной окраски .   язык чистый ,  влажный .  живот мягкий ,   безболезненный во всех отделах .  стул ,  мочеиспускание не нарушены .  | аллергии_неизвестны | нет_анамнеза | нет_осмотра
группа_z номер_00 подномер_ | общий осмотр и обследование лиц ,  не имеющих жалоб или установ ленного диагноза | возраст_48 | пол_2 | общее_состояние_неизвестно | антиб - ки пенициллинового ряда | нет_анамнеза | нет_осмотра
группа_h номер_61 подномер_2 | серная пробка | возраст_33 | пол_2 | общее_состояние_неизвестно | аллергии_неизвестны | нет_анамнеза | общее состояние удовлетворительное .  конфигурация лица не изменена .  кожа лица ,  красная кайма губ нормальной окраски ,  в складку собираются .  регионарные лимфатические узлы не увеличены .  открывание не ограничено ,  движения нижней челюсти свободные . 
группа_k номер_02 подномер_2 | кариес цемента | в

In [30]:
! wc fasttext_valid.txt

   28309  1320306 15931821 fasttext_valid.txt


In [61]:
! fasttext supervised -input fasttext_train.txt -wordNgrams 2 -loss hs -thread 32 -output model_categories -dim 300 #-pretrainedVectors ft_native_300_ru_wiki_lenta_lower_case.vec

/bin/sh: 1: fasttext: not found


In [ ]:
! fasttext test model_categories.bin fasttext_valid.txt

In [ ]:
! fasttext predict-prob model_categories.bin fasttext_test.txt > y_pred_test.txt

In [ ]:
! fasttext predict-prob model_categories.bin fasttext_valid.txt > y_pred_valid.txt

In [31]:
def read_pred(filename):
    with open(filename) as file:
        lines = file.readlines()
        y_pred = np.array([float(line.split()[1]) if line.split()[0] == '__label__1' else 1 - float(line.split()[1])
                           for line in lines])
    return y_pred

In [32]:
y_pred_valid = read_pred('y_pred_valid.txt')

In [33]:
roc_auc_score(valid_dataframe.Revisit, y_pred_valid)

0.6995889037349154

In [37]:
y_pred_test = read_pred('y_pred_test.txt')

In [38]:
submission = pd.read_csv('/root/dataset/RoboMed/sample_submission.csv')

In [39]:
submission.proba = y_pred_test

In [40]:
submission.to_csv('submission_ft.csv', index=False)